In [33]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from scipy.stats import zscore
from scipy.stats import boxcox
import scipy.stats as stats
import torch
%matplotlib inline

In [34]:
idx = 31

In [35]:
import sys
sys.path.append("..") 
sys.path.append("../..") 
import torch
import numpy as np
import torch.nn.functional as F
import json

import hydra
from omegaconf import DictConfig
import paths

In [36]:
import seaborn as sns
sns.set() 
import matplotlib.pyplot as plt
import numpy as np
import torch
import json
import paths

In [37]:
from models.unet_module import DownscalingUnetLightning 
from DownscalingDataModule import DownscalingDataModule

Dataloaders

In [38]:
# Define train and val input/target paths (update file paths as needed)
train_input_paths = {
    'precip': paths.DATASETS_TRAINING_DIR + "/RhiresD_input_train_scaled.nc",
    'temp': paths.DATASETS_TRAINING_DIR + "/TabsD_input_train_scaled.nc",
}
train_target_paths = {
    'precip': paths.DATASETS_TRAINING_DIR + "/RhiresD_target_train_scaled.nc",
    'temp': paths.DATASETS_TRAINING_DIR + "/TabsD_target_train_scaled.nc",

}
val_input_paths = {
    'precip': paths.DATASETS_TRAINING_DIR + "/RhiresD_input_val_scaled.nc",
    'temp': paths.DATASETS_TRAINING_DIR + "/TabsD_input_val_scaled.nc",

}
val_target_paths = {
    'precip': paths.DATASETS_TRAINING_DIR + "/RhiresD_target_val_scaled.nc",
    'temp': paths.DATASETS_TRAINING_DIR + "/TabsD_target_val_scaled.nc",

}

test_input_paths= {
    'precip': paths.DATASETS_TRAINING_DIR + "/RhiresD_input_test_scaled.nc",
    'temp': paths.DATASETS_TRAINING_DIR + "/TabsD_input_test_scaled.nc",

}

test_target_paths = {
    'precip': paths.DATASETS_TRAINING_DIR + "/RhiresD_target_test_scaled.nc",
    'temp': paths.DATASETS_TRAINING_DIR + "/TabsD_target_test_scaled.nc",

}


In [39]:
elevation_path = '/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/elevation.tif'

In [40]:
dm = DownscalingDataModule(
    train_input=train_input_paths,
    train_target=train_target_paths,
    val_input=val_input_paths,
    val_target=val_target_paths,
    test_input=test_input_paths,
    test_target=test_target_paths,
    elevation=elevation_path,
    batch_size=32,
    num_workers=4,
    preprocessing={
        'variables': {
            'input': {
                'precip': 'RhiresD',
                'temp': 'TabsD',

            },
            'target': {
                'precip': 'RhiresD',
                'temp': 'TabsD',

            }
        },
        'preprocessing': {
            'nan_to_num': True,
            'nan_value': 0.0
        }
    }
)
dm.setup()


In [41]:
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

In [42]:
train_inputs, train_targets = next(iter(train_loader))
val_inputs, val_targets = next(iter(val_loader))
test_inputs, test_targets = next(iter(test_loader))

1. Plotting Unet pred for idex = idx

In [43]:
ckpt_unet = paths.LDM_DIR+"/trained_ckpts_optimised/12km/LDM_conditional.models.unet_module.DownscalingUnetLightning_logtransform_lr0.01_precip_loss_weight5.0_1.0_crps[0, 1]_factor0.5_pat3.ckpt.ckpt"

In [44]:

model_UNet = DownscalingUnetLightning(
    in_ch=3,  # 4 vars + elevation
    out_ch=2,  # 4 output variables
    features=[64, 128, 256, 512],
    channel_names=["precip", "temp"],
    
)

In [45]:
unet_state_dict = torch.load(ckpt_unet, map_location="cpu",weights_only=False)["state_dict"]
model_UNet.load_state_dict(unet_state_dict, strict=False)
model_UNet.eval()

DownscalingUnetLightning(
  (unet): DownscalingUnet(
    (encoder): ModuleList()
    (decoder): ModuleList()
    (e1): EncoderBlock(
      (conv): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU()
        )
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (e2): EncoderBlock(
      (conv): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2)

In [46]:
with open(paths.DATASETS_TRAINING_DIR + "/RhiresD_scaling_params.json", 'r') as f:
    pr_params = json.load(f)
with open(paths.DATASETS_TRAINING_DIR + "/TabsD_scaling_params.json", 'r') as f:
    temp_params = json.load(f)

def denorm_pr(x):

    return np.exp(x * pr_params['std'] + pr_params['mean']) - pr_params['epsilon']

def denorm_temp(x, params):

    return x * params['std'] + params['mean']

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [48]:
UNet_model = model_UNet.to(device)

#VAE encoding residuals 

In [49]:
unet_regr= DownscalingUnetLightning(in_ch=3,
                                    out_ch=2,
                                    features=[64, 128, 256, 512],
                                    channel_names=["precip", "temp"])

In [50]:
unet_regr_ckpt = torch.load("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/LDM_conditional/trained_ckpts_optimised/12km/LDM_conditional.models.unet_module.DownscalingUnetLightning_logtransform_lr0.01_precip_loss_weight5.0_1.0_crps[0, 1]_factor0.5_pat3.ckpt.ckpt",
                            map_location="cpu",
weights_only=False)["state_dict"]

In [51]:
unet_regr.load_state_dict(unet_regr_ckpt, strict=False) #Loading it second time for regression network inside VAE
unet_regr.eval()

DownscalingUnetLightning(
  (unet): DownscalingUnet(
    (encoder): ModuleList()
    (decoder): ModuleList()
    (e1): EncoderBlock(
      (conv): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU()
        )
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (e2): EncoderBlock(
      (conv): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2)

Instantiating VAE

Loading LDM ckp

Inspecting ckps

In [52]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

unet_regr = DownscalingUnetLightning(
    in_ch=3,
    out_ch=2,
    features=[64, 128, 256, 512],
    channel_names=["precip", "temp"],
    precip_scaling_json=paths.DATASETS_TRAINING_DIR + "/RhiresD_scaling_params.json",
)
unet_regr_ckpt = torch.load(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/LDM_conditional/trained_ckpts_optimised/12km/LDM_conditional.models.unet_module.DownscalingUnetLightning_logtransform_lr0.01_precip_loss_weight5.0_1.0_crps[0, 1]_factor0.5_pat3.ckpt.ckpt",
    map_location="cpu",weights_only=False
)["state_dict"]
unet_regr.load_state_dict(unet_regr_ckpt, strict=False)
unet_regr = unet_regr.to(device)
unet_regr.eval()

DownscalingUnetLightning(
  (unet): DownscalingUnet(
    (encoder): ModuleList()
    (decoder): ModuleList()
    (e1): EncoderBlock(
      (conv): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (5): ReLU()
        )
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (e2): EncoderBlock(
      (conv): DoubleConv(
        (conv): Sequential(
          (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2)

In [53]:
from models.components.diff.denoiser.unet import UNetModel
from models.components.diff.denoiser.ddim import DDIMSampler
from models.components.diff.conditioner import AFNOConditionerNetCascade
from models.diff_module import DDIMResidualContextual

In [54]:
# Instantiate denoiser (pixel space, 2 channels, config-matched)
denoiser = UNetModel(
    model_channels=32,
    in_channels=2,
    out_channels=2,
    num_res_blocks=2,
    attention_resolutions=[1, 2, 4],
    context_ch=[32, 64, 128],
    channel_mult=[1, 2, 4],
    conv_resample=True,
    dims=2,
    use_fp16=False,
    num_heads=2
)

# Instantiate conditioner (no autoencoder, pixel context)
conditioner = AFNOConditionerNetCascade(
    autoencoder=None,
    input_channels=[2],
    embed_dim=[32, 64, 128],
    analysis_depth=3,
    cascade_depth=3,
    context_ch=[32, 64, 128]
)




ddim = DDIMResidualContextual(
    denoiser=denoiser,
    context_encoder=conditioner,
    timesteps=1000,
    parameterization="eps",
    loss_type="l2"
)



ldm_ckpt = torch.load(
    "/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/DDIM_conditional_derived/trained_ckpts/12km/DDIM_checkpoint_model.parameterization=0_model.timesteps=0_model.beta_schedule=0.ckpt",
    map_location=device
)


ddim.load_state_dict(ldm_ckpt["state_dict"], strict=False)
ddim = ddim.to(device)
ddim.eval()

normalization called with channels=32, norm_type=group, norm_kwargs={'num_groups': 8}
normalization called with channels=64, norm_type=group, norm_kwargs={'num_groups': 8}
normalization called with channels=64, norm_type=group, norm_kwargs={'num_groups': 8}
normalization called with channels=128, norm_type=group, norm_kwargs={'num_groups': 8}


DDIMResidualContextual(
  (denoiser): UNetModel(
    (time_embed): Sequential(
      (0): Linear(in_features=32, out_features=128, bias=True)
      (1): SiLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
    (input_blocks): ModuleList(
      (0): TimestepEmbedSequential(
        (0): Conv2d(2, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (1-2): 2 x TimestepEmbedSequential(
        (0): ResBlock(
          (in_layers): Sequential(
            (0): Identity()
            (1): SiLU()
            (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )
          (emb_layers): Sequential(
            (0): SiLU()
            (1): Linear(in_features=128, out_features=32, bias=True)
          )
          (out_layers): Sequential(
            (0): Identity()
            (1): SiLU()
            (2): Dropout(p=0, inplace=False)
            (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          )


In [55]:
sampler = DDIMSampler(ddim, device=device)

In [56]:
sampler

In [ ]:
with torch.no_grad():
    input_sample = test_inputs[idx].unsqueeze(0).to(device)  # (1, C_in, H, W)
    unet_pred = unet_regr(input_sample)                      # (1, C_out, H, W)
    context = [(unet_pred, None)]
    sample_shape = unet_pred.shape[1:]                       # (C_out, H, W)
    z = torch.randn((1, *sample_shape), device=device)       # (1, C_out, H, W)

    # Dsampling
    residual, _ = sampler.sample(
        S=1000,
        batch_size=1,
        shape=sample_shape,
        conditioning=context,
        eta=0.0,
        verbose=True,
        x_T=z,
    )
    final_pred = unet_pred + residual

    final_pred_np = final_pred[0].cpu().numpy()              # (C_out, H, W)
    unet_pred_np = unet_pred[0].cpu().numpy()                # (C_out, H, W)
    input_np = input_sample[0, :unet_pred_np.shape[0]].cpu().numpy()  # (C_out, H, W)
    target_np = test_targets[idx][:unet_pred_np.shape[0]].cpu().numpy()  # (C_out, H, W)

    channel_names = ["Precip", "Temp"]
    params_list = [pr_params, temp_params]

    input_denorm = np.empty_like(input_np)
    for i, params in enumerate(params_list):
        input_denorm[i] = denorm_pr(input_np[i]) if i == 0 else denorm_temp(input_np[i], params)

    unet_pred_denorm = np.empty_like(unet_pred_np)
    for i, params in enumerate(params_list):
        unet_pred_denorm[i] = denorm_pr(unet_pred_np[i]) if i == 0 else denorm_temp(unet_pred_np[i], params)

    ldm_pred_denorm = np.empty_like(final_pred_np)
    for i, params in enumerate(params_list):
        ldm_pred_denorm[i] = denorm_pr(final_pred_np[i]) if i == 0 else denorm_temp(final_pred_np[i], params)

    target_denorm = np.empty_like(target_np)
    for i, params in enumerate(params_list):
        target_denorm[i] = denorm_pr(target_np[i]) if i == 0 else denorm_temp(target_np[i], params)

    # Debug: Print min/max to check for NaNs or constant arrays
    print("input_denorm", np.nanmin(input_denorm), np.nanmax(input_denorm))
    print("unet_pred_denorm", np.nanmin(unet_pred_denorm), np.nanmax(unet_pred_denorm))
    print("ldm_pred_denorm", np.nanmin(ldm_pred_denorm), np.nanmax(ldm_pred_denorm))
    print("target_denorm", np.nanmin(target_denorm), np.nanmax(target_denorm))

    vmins = [min(input_denorm[j].min(), unet_pred_denorm[j].min(), ldm_pred_denorm[j].min(), target_denorm[j].min()) for j in range(len(params_list))]
    vmaxs = [max(input_denorm[j].max(), unet_pred_denorm[j].max(), ldm_pred_denorm[j].max(), target_denorm[j].max()) for j in range(len(params_list))]

    # Plot
    fig, axes = plt.subplots(4, len(params_list), figsize=(5*len(params_list), 12), dpi=150)
    if len(params_list) == 1:
        axes = axes[:, np.newaxis]
    for j in range(len(params_list)):
        axes[0, j].imshow(np.flipud(input_denorm[j]), cmap='coolwarm', vmin=vmins[j], vmax=vmaxs[j])
        axes[0, j].set_title(f"Input (denorm) {channel_names[j]}")
        axes[0, j].axis('off')
        axes[1, j].imshow(np.flipud(unet_pred_denorm[j]), cmap='coolwarm', vmin=vmins[j], vmax=vmaxs[j])
        axes[1, j].set_title(f"UNet Output (denorm) {channel_names[j]}")
        axes[1, j].axis('off')
        axes[2, j].imshow(np.flipud(ldm_pred_denorm[j]), cmap='coolwarm', vmin=vmins[j], vmax=vmaxs[j])
        axes[2, j].set_title(f"LDM Output (denorm) {channel_names[j]}")
        axes[2, j].axis('off')
        axes[3, j].imshow(np.flipud(target_denorm[j]), cmap='coolwarm', vmin=vmins[j], vmax=vmaxs[j])
        axes[3, j].set_title(f"Ground Truth (denorm) {channel_names[j]}")
        axes[3, j].axis('off')
        cbar = fig.colorbar(axes[0, j].images[0], ax=axes[:, j], fraction=0.02, pad=0.01)
        cbar.ax.set_ylabel(channel_names[j])

    # Save to file in case notebook output is not working
    fig.savefig("/work/FAC/FGSE/IDYST/tbeucler/downscaling/sasthana/Downscaling/Downscaling_Models/DDIM_conditional_derived/outputs/debug_output.png")
    print("Plot saved as debug_output.png")

plt.show()

Selected timesteps for ddim sampler: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 2

DDIM Sampler:   0%|          | 0/999 [00:00<?, ?it/s]

DDIM Sampler:   0%|          | 2/999 [00:00<01:19, 12.50it/s]

Conditioner cascade[0] shape after proj: torch.Size([1, 32, 240, 370])
Conditioner cascade[1] shape after proj: torch.Size([1, 64, 120, 185])
Conditioner cascade[2] shape after proj: torch.Size([1, 128, 60, 92])
UNet forward called with x shape: torch.Size([1, 2, 240, 370]), timesteps shape: torch.Size([1])
UNet forward context[0] shape: torch.Size([1, 32, 240, 370])
UNet forward context[1] shape: torch.Size([1, 64, 120, 185])
UNet forward context[2] shape: torch.Size([1, 128, 60, 92])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 120, 185, 64])
AFNO y (after proj/interp) shape: torch.Size([1, 120, 185, 64])
AFNO norm1(x) shape: torch.Size([1, 120, 185, 64])
AFNO y (after proj/interp) shape: torch.Size([1, 120, 185, 64])
AFNO norm1(x) shape: torch.Size([1, 60, 93, 128])

DDIM Sampler:   0%|          | 4/999 [00:00<01:19, 12.54it/s]

...

DDIM Sampler:  52%|█████▏    | 524/999 [00:32<00:29, 16.00it/s]

AFNO norm1(x) shape: torch.Size([1, 60, 93, 128])
AFNO y (after proj/interp) shape: torch.Size([1, 60, 93, 128])
AFNO norm1(x) shape: torch.Size([1, 60, 93, 128])
AFNO y (after proj/interp) shape: torch.Size([1, 60, 93, 128])
AFNO norm1(x) shape: torch.Size([1, 60, 93, 128])
AFNO y (after proj/interp) shape: torch.Size([1, 60, 93, 128])
AFNO norm1(x) shape: torch.Size([1, 120, 185, 64])
AFNO y (after proj/interp) shape: torch.Size([1, 120, 185, 64])
AFNO norm1(x) shape: torch.Size([1, 120, 185, 64])
AFNO y (after proj/interp) shape: torch.Size([1, 120, 185, 64])
AFNO norm1(x) shape: torch.Size([1, 120, 185, 64])
AFNO y (after proj/interp) shape: torch.Size([1, 120, 185, 64])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: t

DDIM Sampler:  53%|█████▎    | 528/999 [00:33<00:29, 15.99it/s]

AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
Conditioner cascade[0] shape after proj: torch.Size([1, 32, 240, 370])
Conditioner cascade[1] shape after proj: torch.Size([1, 64, 120, 185])
Conditioner cascade[2] shape after proj: torch.Size([1, 128, 60, 92])
UNet forward called with x shape: torch.Size([1, 2, 240, 370]), timesteps shape: torch.Size([1])
UNet forward context[0] shape: torch.Size([1, 32, 240, 370])
UNet forward context[1] shape: torch.Size([1, 64, 120, 185])
UNet forward context[2] shape: torch.Size([1, 128, 60, 92])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 240, 370, 32])
AFNO y (after proj/interp) shape: torch.Size([1, 240, 370, 32])
AFNO norm1(x) shape: torch.Size([1, 120, 185, 64]

DDIM Sampler:  53%|█████▎    | 530/999 [00:33<00:29, 15.99it/s]